In [151]:
import requests
import json
import os
import pandas as pd
from functools import reduce
import operator
import pyjsonviewer
import geopandas as gpd
import time
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Point
from pymongo import MongoClient
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from pymongo import GEOSPHERE
import shapely.geometry

from dotenv import load_dotenv
load_dotenv()

True

## Funciones útiles

In [14]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [15]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [16]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [data["latt"], data["longt"]]}
    except:
        return data

In [299]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
geospatial = db.get_collection("geospatial")

## Lista de categorías e id de Foursquare
- scrapping

In [17]:
def get_categories():
    url = "https://developer.foursquare.com/docs/build-with-foursquare/categories/"
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    venue_cat = soup.find_all("ul", {"class": "VenueCategories__Wrapper-sc-15dn453-0 dmcDKQ"})
    cat = {}
    li = venue_cat[0].find_all("li")
    for l in li:
        cat[l.find("h3").getText()] = l.find("p").getText()
        
    return pd.DataFrame.from_dict(cat,orient="Index").reset_index().rename(columns={"index": "category", 0: "categoryId"})

categories = get_categories()

In [18]:
categories.sample(3)

,category,categoryId
297,Ouzeri,53d6c1b0e4b02351e88a83dc
304,Poke Place,5bae9231bedf3950379f89d4
11,Disc Golf,52e81612bcbc57f1066b79e8


In [19]:
def categories_dic(cat_list):    
    dicc = {}
    for c in cat_list:
        dicc[c] = categories.categoryId[categories["category"] == c].values[0]
    return dicc

In [20]:
venues_list = ["Design Studio","Nursery School","Preschool","Elementary School","Tech Startup","Coffee Shop",
               "Airport Terminal","Train Station",
              "Nightlife Spot","Vegetarian / Vegan Restaurant","Basketball Stadium",
               "Veterinarian","Pet Service","Pet Store"]  

In [21]:
cat_dic = categories_dic(venues_list)
cat_dic

{'Design Studio': '4bf58dd8d48988d1f4941735',
 'Nursery School': '4f4533814b9074f6e4fb0107',
 'Preschool': '52e81612bcbc57f1066b7a45',
 'Elementary School': '4f4533804b9074f6e4fb0105',
 'Tech Startup': '4bf58dd8d48988d125941735',
 'Coffee Shop': '4bf58dd8d48988d1e0931735',
 'Airport Terminal': '4bf58dd8d48988d1eb931735',
 'Train Station': '4bf58dd8d48988d129951735',
 'Nightlife Spot': '4d4b7105d754a06376d81259',
 'Vegetarian / Vegan Restaurant': '4bf58dd8d48988d1d3941735',
 'Basketball Stadium': '4bf58dd8d48988d18b941735',
 'Veterinarian': '4d954af4a243a5684765b473',
 'Pet Service': '5032897c91d4c4b30a586d69',
 'Pet Store': '4bf58dd8d48988d100951735'}

In [22]:
def localizations_coor(dicc):
    local = {}
    for city,loc in localizations_dicc.items():
        local[city] = geocode(loc)["coordinates"]
        time.sleep(15)
    return local
    

In [23]:
# localizations_dicc={"Austin":"Texas Capitol, Austin, Texas","Hamburg":"Hamburg city center,Hamburg,Germany","Newcastle":"Newcastle upon Tyne, United Kingdom"}
# localizations = localizations_coor(localizations_dicc)
# localizations

In [24]:
 localizations = {'Austin': ['30.27125853189173', '-97.75551842126049'],
 'Hamburg': ['53.55687309076416', '10.002900693677232'],
 'Newcastle': ['54.96923892979405', '-1.6162802662146134']}

In [25]:
def place_store_dicc(localizations,cat_dic):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    client_id = os.getenv("four")
    client_secret = os.getenv("foursec")
    
    venues_dic = {}
    aus_dic ={}
    ham_dic = {}
    ncas_dic ={}
    for loc,coor in localizations.items():
        for cat,cat_id in cat_dic.items():
            time.sleep(1)
            parametros={
                "client_id": client_id,
                "client_secret": client_secret,
                "v": "20180323",
                "ll": f"{coor[0]}, {coor[1]}",
                "categoryId":cat_id,
                "radius":2000
            }
            if loc == "Austin":
                aus_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"] 
                if cat == "Pet Store":
                    print(f"{loc} done")
            elif loc == "Hamburg":                
                ham_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"]
                if cat == "Pet Store":
                    print(f"{loc} done")
            elif loc == "Newcastle":                
                ncas_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"] 
                if cat == "Pet Store":
                    print(f"{loc} done")
            
    venues_dic = {"Austin":aus_dic,
                 "Hamburg":ham_dic,
                  "Newcastle": ncas_dic
                 }

    return venues_dic


In [26]:
total_venues = place_store_dicc(localizations,cat_dic)

Austin done
Hamburg done
Newcastle done


In [49]:
with open('../data/total_venues.json', 'w') as f:
    json.dump(total_venues, f)

In [35]:
def extraetodo(json):
    total = []
    for city,categories in json.items():
        for cat,value in categories.items():
            try:
                todo = {"nombre": ["name"],"sub_category":["categories",0,"name"],
                        "latitud": ["location", "lat"], "longitud": ["location", "lng"],
                        "distance":["location","distance"]}

                for elemento in value:
                    store = {key: getFromDict(elemento,value) for key,value in todo.items()}
                    store["location"] = type_point([store["latitud"], store["longitud"]])
                    store["city"] = city
                    store["category"] = cat
                    total.append(store)

                #print(f"{elemento} done")
            except:

                print(f"{value.index(elemento)} wrong")

    return total

In [305]:
venues = extraetodo(total_venues)

In [306]:
venues_df = pd.DataFrame(venues)

In [257]:
venues_df.sample(3)

,nombre,sub_category,latitud,longitud,distance,location,city,category
37,Can We Studios,Tech Startup,30.267561,-97.740792,1474,"{'type': 'Point', 'coordinates': [30.267560747...",Austin,Tech Startup
104,Firehouse Hostel and Lounge,Lounge,30.268096,-97.741283,1413,"{'type': 'Point', 'coordinates': [30.268096341...",Austin,Nightlife Spot
195,Kita Hinrichsenstrasse,Daycare,53.558216,10.033161,2006,"{'type': 'Point', 'coordinates': [53.558215930...",Hamburg,Preschool


In [258]:
#borrar todos los cafés que no sean starbucks
no_starbucks = list(venues_df.index[(venues_df["category"] == "Coffee Shop") & (venues_df["nombre"] != "Starbucks")])
venues_df.drop(no_starbucks,axis=0, inplace = True)

In [260]:
venues_df.reset_index(drop = True,inplace = True)

In [282]:
venues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   nombre        414 non-null    object  
 1   sub_category  414 non-null    object  
 2   latitud       414 non-null    float64 
 3   longitud      414 non-null    float64 
 4   distance      414 non-null    int64   
 5   location      414 non-null    object  
 6   city          414 non-null    object  
 7   category      414 non-null    object  
 8   geometry      414 non-null    geometry
dtypes: float64(2), geometry(1), int64(1), object(5)
memory usage: 29.2+ KB


In [262]:
venues_df.to_csv("../data/venues_df.csv",index = False)

In [43]:
Austin_df = venues_df[venues_df["city"] == "Austin"].sort_values(["category","distance"])
Austin_df.head(3)

,nombre,sub_category,latitud,longitud,distance,location,city,category
134,Texas School For The Deaf Basketball Gym,Basketball Stadium,30.253043,-97.752368,2050,"{'type': 'Point', 'coordinates': [30.253042851...",Austin,Basketball Stadium
56,Starbucks,Coffee Shop,30.269783,-97.753914,225,"{'type': 'Point', 'coordinates': [30.269783, -...",Austin,Coffee Shop
79,Starbucks,Coffee Shop,30.265858,-97.746308,1070,"{'type': 'Point', 'coordinates': [30.265857567...",Austin,Coffee Shop


In [44]:
Hamburg_df = venues_df[venues_df["city"] == "Hamburg"].sort_values(["category","distance"])
Hamburg_df.head(3)

,nombre,sub_category,latitud,longitud,distance,location,city,category
239,Starbucks,Coffee Shop,53.552863,10.005558,479,"{'type': 'Point', 'coordinates': [53.552863, 1...",Hamburg,Coffee Shop
247,Starbucks,Coffee Shop,53.551256,10.000930,638,"{'type': 'Point', 'coordinates': [53.551256304...",Hamburg,Coffee Shop
241,Starbucks,Coffee Shop,53.555068,9.990986,813,"{'type': 'Point', 'coordinates': [53.555067825...",Hamburg,Coffee Shop


In [130]:
Newcastle_df = venues_df[venues_df["city"] == "Newcastle"].sort_values(["category","distance"])
Newcastle_df.sample()

,nombre,sub_category,latitud,longitud,distance,location,city,category,geometry
385,Fluid Pixel Offices,Tech Startup,54.972701,-1.608061,651,"{'type': 'Point', 'coordinates': [54.972700706...",Newcastle,Tech Startup,POINT (54.97270 -1.60806)


In [127]:
Austin_df.groupby("category").agg({"distance":"mean"}).reset_index().head(3)

,category,distance
0,Basketball Stadium,2050.000
1,Coffee Shop,1306.375
2,Design Studio,1199.500


In [128]:
Hamburg_df.groupby("category").agg({"distance":"mean"}).reset_index().head(3)

,category,distance
0,Coffee Shop,643.333333
1,Design Studio,1455.250000
2,Elementary School,2336.000000


In [129]:
Newcastle_df.groupby("category").agg({"distance":"mean"}).reset_index().head(3)

,category,distance
0,Basketball Stadium,1183.000000
1,Coffee Shop,603.500000
2,Design Studio,862.842105


In [56]:
venues_Austin = total_venues["Austin"]

In [122]:
with open('../data/venues_Austin.json', 'w') as f:
    json.dump(venues_Austin, f)

In [58]:
venues_Hamburg = total_venues["Hamburg"]
with open('../data/venues_Hamburg.json', 'w') as f:
    json.dump(venues_Hamburg, f)

In [59]:
venues_Newcastle = total_venues["Newcastle"]
with open('../data/Newcastle.json', 'w') as f:
    json.dump(venues_Hamburg, f)

In [338]:
geo_venues = gpd.GeoDataFrame(venues_df, geometry=gpd.points_from_xy(venues_df.longitud,venues_df.latitud))

In [339]:
geo_venues.drop("location", axis=1, inplace = True)


In [340]:
geo_venues.sample(2)

,nombre,sub_category,latitud,longitud,distance,city,category,geometry
189,Kita Kohlhöfen,Nursery School,53.553187,9.979998,1569,Hamburg,Nursery School,POINT (9.98000 53.55319)
296,Ta Vegan House,Vegetarian / Vegan Restaurant,53.545310,9.975342,2231,Hamburg,Vegetarian / Vegan Restaurant,POINT (9.97534 53.54531)


In [341]:
geo_venues['geometry']=geo_venues['geometry'].apply(lambda x:shapely.geometry.mapping(x))

/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [343]:
geo_venus_dic = geo_venues.to_dict(orient="records")

In [344]:
geospatial.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [345]:
geospatial.insert_many(geo_venus_dic)